In [1]:
import collections
import json
from requests_html import HTMLSession
from bs4 import BeautifulSoup

from urllib.request import urlopen, Request
from fake_useragent import UserAgent

import pandas as pd

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from dateutil.parser import parse
import csv

from datetime import datetime
import time

from gensim.models import Word2Vec
from konlpy.tag import Okt
from soynlp.normalizer import *
# from collections import Counter
import re



In [2]:
texts_article = []
time_zone = []
comments = []

texts_title = []
texts_url = []

page_url = [1]

commentss = ''


In [4]:
def tokenize(text):
    p = re.compile("[0-9]+")
    text = p.sub(" ", text)
    p = re.compile("\W+")
    text = p.sub(" ", text)
    p = re.compile("_")
    text = p.sub(" ", text)

    okt = Okt()
    pos = okt.pos(text)
    # print(pos)
    comment = []

    for index_row in range(len(pos)):
        if(pos[index_row][1] == 'Noun'):
            if index_row+1 < len(pos):
                if pos[index_row+1][1] == 'Suffix':
                    temp = pos[index_row+1][0]
                    temp_string = pos[index_row][0] + temp
                    comment.append(temp_string)
                if pos[index_row+1][1] == 'Modifier':
                    if index_row+2 < len(pos):
                        if pos[index_row+2][1] == 'Noun':
                            temp_string = pos[index_row][0] + \
                                pos[index_row+1][0] + pos[index_row+2][0]
                            comment.append(temp_string)
                else:
                    comment.append(pos[index_row][0])
            else:
                comment.append(pos[index_row][0])

        if pos[index_row][1] == 'Modifier':
            if index_row+1 < len(pos):
                if pos[index_row+1][1] == 'Noun':
                    temp_string = pos[index_row][0] + pos[index_row+1][0]
                    comment.append(temp_string)

        if(pos[index_row][1] == 'VerbPrefix'):
            if index_row+1 < len(pos):
                temp = pos[index_row+1][0]
                temp_string = pos[index_row][0] + temp
                comment.append(temp_string)

        if(pos[index_row][1] == 'Verb'):
            if index_row-1 >= 0:
                if pos[index_row-1][1] != 'VerbPrefix':
                    comment.append(pos[index_row][0])

        if(pos[index_row][1] == 'Adjective'):
            comment.append(pos[index_row][0])

        # if(pos[index_row][1] == 'Verb'):
        #     comment.append(pos[index_row][0])
    comments = ' '.join(comment)
    return comments


def crawling_url(url):
    global texts_url
    global page_url
    session = HTMLSession()
    res = session.get(url)

    soup = BeautifulSoup(res.content, 'html.parser')
    urls = soup.find_all('a')
    for x in range(len(urls)):
        if urls[x].text:
            check = urls[x]['href']
            if 'mid=sec' in check:
                if 'oid=' in check:
                    if 'news.naver.com' in check:
                        if 'aid=' in check:
                            if 'isYeonhapFlash=Y' in check:
                                texts_url.append(check)


def crawling_url_cluster(url):
    global texts_url
    session = HTMLSession()
    res = session.get(url)

    soup = BeautifulSoup(res.content, 'html.parser')
    urls = soup.find_all('a', attrs={'class': 'nclicks(cls_pol.clsart1)'})
    if urls:
        for x in range(len(urls)):
            if urls[x].text:
                texts_url.append(urls[x]['href'])
    else:
        print("No such tag")

    # return url


def find_Maxpage(url, a):
    global page_url
    session = HTMLSession()
    res = session.get(url)

    soup = BeautifulSoup(res.content, 'html.parser')
    urls = soup.find_all('a', attrs={'class': 'nclicks(fls.page)'})
    if a > 0:
        page_url.append(a*10+1)
    if urls:
        for x in range(len(urls)):
            check = urls[x].text
            if check != '다음':
                if check != '이전':
                    page_url.append(check)
            else:
                a = a+1
                next_page = 'https://news.naver.com/main/list.nhn' + \
                    urls[x]['href']
                find_Maxpage(next_page, a)
    else:
        print("No such tag")
    # time.sleep(0.5)

    # return url


def crawling_article(url):
    session = HTMLSession()
    res = session.get(url)

    soup = BeautifulSoup(res.content, 'html.parser')
    article = soup.find('div', attrs={'id': 'articleBodyContents'})
    title = soup.find('h3', attrs={'id': 'articleTitle'})
    if article:
        global texts_article
        global texts_title

        texts_title.append(title.text)
        texts_article.append(article.text)
    else:
        print("No such tag")

    # return article


def getJson_NewsInfo(resp):
    x = 0
    while(resp[x] != '{'):
        x += 1
     # print(response[x:])

    z = len(resp)-1
    while(resp[z] != '}'):
        z -= 1
    # print(response[x:y]+'}')

    rank_json = json.loads(resp[x:z]+'}')

    return rank_json


today = datetime.now().strftime('%Y%m%d')
find_Maxpage('https://news.naver.com/main/list.nhn?mode=LPOD&sid2=140&sid1=001&mid=sec&oid=001&isYeonhapFlash=Y&date={}&page=1'.format(today), 0)
print(page_url)

for x in range(len(page_url)):
    crawling_url(
        'https://news.naver.com/main/list.nhn?mode=LPOD&sid2=140&sid1=001&mid=sec&oid=001&isYeonhapFlash=Y&date={}&page={}'.format(today, page_url[x]))

# today = datetime.now().strftime('%Y%m%d')
# find_Maxpage('https://news.naver.com/main/list.nhn?mode=LPOD&sid2=140&sid1=001&mid=sec&oid=001&isYeonhapFlash=Y&date=20210406&page=1', 0)
# print(page_url)

# for x in range(len(page_url)):
#     crawling_url(
#         'https://news.naver.com/main/list.nhn?mode=LPOD&sid2=140&sid1=001&mid=sec&oid=001&isYeonhapFlash=Y&date=20210406&page={}'.format(page_url[x]))


texts_url = set(texts_url)
texts_url = list(texts_url)

# print("*********url 정보***********")
print(texts_url)
print(len(texts_url))


oid = []
aid = []
headers = []
com_url = []


def getOid_Aid(uri):
    global oid
    global aid
    oid.append(uri.split("oid=")[1].split("&")[0])
    aid.append(uri.split("aid=")[1].split("&is")[0])




[1, '2', '3', '4', '5', '6', '7', '8', '9', '10', 11, '12', '13', '14', '15', '2', '3', '4', '5', '6', '7', '8', '9', '10', 11, '12', '13', '14', '15']
['https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012397378&isYeonhapFlash=Y&rc=N', 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012399189&isYeonhapFlash=Y&rc=N', 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012399390&isYeonhapFlash=Y&rc=N', 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012398204&isYeonhapFlash=Y&rc=N', 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012397190&isYeonhapFlash=Y&rc=N', 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012398078&isYeonhapFlash=Y&rc=N', 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012398237&isYeonhapFlash=Y&rc=N', 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012398045&isYeonhapFlash=Y&rc=N', 'https://news.naver.com

In [ ]:
for y in range(len(texts_url)):
    a = texts_url[y]
    getOid_Aid(a)
    # oid.append(texts_url[y].split("oid=")[1].split("&")[0])
    # aid.append(texts_url[y].split("aid=")[1].split("&is")[0])

useragent = UserAgent()

for x in range(len(texts_url)):
    comps_url = []
    for y in range(1, 30):
        comps_url.append("https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_politics_m1&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" +
                         oid[x] + "%2C" + aid[x] + "&categoryId=&pageSize=100&indexSize=100&groupId=&listType=OBJECT&pageType=more&page=" + str(y) + "&refresh=false&sort=FAVORITE")
    com_url.append(comps_url)
# print(com_url[0][1])

for v in range(len(texts_url)):
    headers.append({
        'referer': texts_url[v],
        'User-Agent': useragent.chrome
    })

for v in range(len(texts_url)):
    comments_each_news = []
    total_num_comments = 0
    for y in range(len(com_url[v])):
        # print("v:")
        # print(v)
        # print("Y:")
        # print(y)
        resp = urlopen(
            Request(com_url[v][y], headers=headers[v])).read().decode('utf-8')
        # print(resp)
        # time.sleep(0.15)
        if not '"commentList":[]' in resp:
            rank_json = getJson_NewsInfo(resp)

            if rank_json['result']['commentList']:
                for w in range(len(rank_json['result']['commentList'])):
                    temp_1 = tokenize(
                        rank_json['result']['commentList'][w]['contents'])
                    temp_2 = temp_1.split(' ')
                    comments_each_news.append(temp_2)
                    total_num_comments = total_num_comments + 1
                    # print(total_num_comments)
                    # commentss = commentss + \
                    #     rank_json['result']['commentList'][w]['contents']
        else:
            break
    # print("completed : ", total_num_comments)
    if total_num_comments >= 100:
        comments.append(comments_each_news)
    # print(comments[v])

# print(comments[0])
# print(comments[1])

# b = []
# a = [['안녕', '나는', '주민'], ['안녕', '나는', '주민', '주민']]
# for x in range(len(a)):
#     b.append(' '.join(a[x]))
# d = ' '.join(b)
# e = d.split(' ')
# c = collections.Counter(e)
# # print(b)
# print(c.most_common(1)[0][0])
print(len(comments))



In [ ]:
for idx_url in range(len(comments)):
    temp_3 = []
    word2vec_model = Word2Vec(
        comments[idx_url], window=5, size=10000, min_count=13)
    for x in range(len(comments[idx_url])):
        temp_3.append(' '.join(comments[idx_url][x]))
    stringChanged = ' '.join(temp_3)
    stringSplited = stringChanged.split(' ')
    print(idx_url)
    counts = collections.Counter(stringSplited)
    print(counts.most_common(1)[0][0])
    print(word2vec_model.most_similar_cosmul(counts.most_common(1)[0][0]))

#* word Embedding Process(word2vec)
